In [1]:
import pandas as pd
import numpy as np

import seaborn as sns
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd

Mounted at /content/drive


In [2]:
import pandas as pd

# Assuming your file is in Excel format, use read_excel
data = pd.read_csv('/content/drive/MyDrive/lithium_pro/lith_circularity4.csv')
data.head()

,Year,scenario.percent.repurposed,scenario.reuse.lifespan,material,kWh.retired,kg.retired,kg.recycled.weight,kWh,kg.demand,circularity (%),collection_rate,recycling.process_direct,recycling.process_hydro,recycling.process_pyro,Sales.Scenario_SDS,Sales.Scenario_STEPS,Cathode.Scenario_LFP,Cathode.Scenario_NCX
0,2020,0,0,0,2417264.362,7192595.151,6473335.636,20735767.32,68338836.46,0.094724,0.6,1,0,0,1,0,1,0
1,2020,0,0,0,2417264.362,7192595.151,6473335.636,20735767.32,68338836.46,0.094724,0.6,1,0,0,0,1,1,0
2,2020,0,0,0,2417264.362,7192595.151,6473335.636,20735767.32,68338836.46,0.094724,0.6,0,1,0,1,0,1,0
3,2020,0,0,0,2417264.362,7192595.151,6473335.636,20735767.32,68338836.46,0.094724,0.6,0,1,0,0,1,1,0
4,2020,0,0,0,2417264.362,7192595.151,0.000,20735767.32,68338836.46,0.000000,0.6,0,0,1,1,0,1,0


In [3]:
X = data.drop('circularity (%)', axis=1)
y = data['circularity (%)']


print(X,y)

        Year  scenario.percent.repurposed  scenario.reuse.lifespan  material  \
0       2020                            0                        0         0   
1       2020                            0                        0         0   
2       2020                            0                        0         0   
3       2020                            0                        0         0   
4       2020                            0                        0         0   
...      ...                          ...                      ...       ...   
146798  2050                            2                        2         5   
146799  2050                            2                        2         5   
146800  2050                            2                        2         6   
146801  2050                            2                        2         6   
146802  2050                            2                        2         6   

         kWh.retired    kg.retired  kg.

In [4]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Split the data into train, test, and validation sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)


In [5]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LassoCV
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

# Ensure X_train is a DataFrame
if isinstance(X_train, np.ndarray):
    X_train = pd.DataFrame(X_train)
    X_test = pd.DataFrame(X_test)

# Step 1: Feature Selection using Stepwise Regression (LassoCV)
lasso = LassoCV(cv=5, max_iter=10000, random_state=42)  # Increased iterations
lasso.fit(X_train, y_train)

# Select features where coefficients are non-zero
selected_indices = np.where(lasso.coef_ != 0)[0]  # Get indices of non-zero coefficients
X_train_selected = X_train.iloc[:, selected_indices]
X_test_selected = X_test.iloc[:, selected_indices]

# Step 2: Scale features for Neural Network
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_selected)
X_test_scaled = scaler.transform(X_test_selected)

# Step 3: Build & Train Backpropagation Neural Network (MLP)
mlp_model = MLPRegressor(hidden_layer_sizes=(100, 50, 25), activation='relu', solver='adam',
                         learning_rate_init=0.001, max_iter=1000, random_state=42)

mlp_model.fit(X_train_scaled, y_train)

# Step 4: Predictions
y_pred = mlp_model.predict(X_test_scaled)

# Step 5: Model Evaluation
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

# **Fixed RMSLE Calculation**
y_pred_safe = np.maximum(y_pred, 1e-9)  # Avoid log(0)
y_test_safe = np.maximum(y_test, 1e-9)
rmsle = np.sqrt(np.mean((np.log1p(y_pred_safe) - np.log1p(y_test_safe)) ** 2))

# **Fixed MAPE Calculation**
y_test_safe_mape = np.maximum(np.abs(y_test), 1e-2)  # Avoid division by zero
mape = np.mean(np.abs((y_test - y_pred) / y_test_safe_mape)) * 100

# Print results
print(f"SR-BPNN Regression Results:")
print(f"Mean Absolute Error (MAE): {mae:.2f}")
print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")
print(f"R-squared (R2): {r2:.2f}")
print(f"Root Mean Squared Logarithmic Error (RMSLE): {rmsle:.2f}")
print(f"Mean Absolute Percentage Error (MAPE): {mape:.2f}%")


SR-BPNN Regression Results:
Mean Absolute Error (MAE): 0.01
Mean Squared Error (MSE): 0.00
Root Mean Squared Error (RMSE): 0.01
R-squared (R2): 0.98
Root Mean Squared Logarithmic Error (RMSLE): 0.01
Mean Absolute Percentage Error (MAPE): 17.39%


In [6]:
import pandas as pd

# Assuming your file is in Excel format, use read_excel
data = pd.read_csv('/content/drive/MyDrive/lithium_pro/lith_cost4.csv')
data.head()

,Category,Output,material,Year,Recycling Throughput,Cathode Scenario,Location scenario,Type_All,Type_Direct,Type_Hydro,Type_Pyro,Location of recycling_China,Location of recycling_U.S.
0,5,2.546235,0,2020,10000,1,1,0,0,0,1,0,1
1,5,2.051076,0,2020,10000,1,1,0,0,1,0,0,1
2,5,3.488244,0,2020,10000,1,1,0,1,0,0,0,1
3,8,0.099729,9,2020,10000,1,1,0,0,0,1,0,1
4,8,0.327106,27,2020,10000,1,1,0,0,0,1,0,1


In [7]:
X = data.drop('Output', axis=1)
y = data['Output']


print(X,y)

        Category  material  Year  Recycling Throughput  Cathode Scenario  \
0              5         0  2020                 10000                 1   
1              5         0  2020                 10000                 1   
2              5         0  2020                 10000                 1   
3              8         9  2020                 10000                 1   
4              8        27  2020                 10000                 1   
...          ...       ...   ...                   ...               ...   
166279         7        33  2050                  9000                 2   
166280         7         2  2050                  9000                 2   
166281         7        28  2050                  9000                 2   
166282         7         3  2050                  9000                 2   
166283         7        19  2050                  9000                 2   

        Location scenario  Type_All  Type_Direct  Type_Hydro  Type_Pyro  \
0           

In [8]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Split the data into train, test, and validation sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)


In [9]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LassoCV
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

# Ensure X_train is a DataFrame
if isinstance(X_train, np.ndarray):
    X_train = pd.DataFrame(X_train)
    X_test = pd.DataFrame(X_test)

# Step 1: Feature Selection using Stepwise Regression (LassoCV)
lasso = LassoCV(cv=5, max_iter=10000, random_state=42)  # Increased iterations
lasso.fit(X_train, y_train)

# Select features where coefficients are non-zero
selected_indices = np.where(lasso.coef_ != 0)[0]  # Get indices of non-zero coefficients
X_train_selected = X_train.iloc[:, selected_indices]
X_test_selected = X_test.iloc[:, selected_indices]

# Step 2: Scale features for Neural Network
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_selected)
X_test_scaled = scaler.transform(X_test_selected)

# Step 3: Build & Train Backpropagation Neural Network (MLP)
mlp_model = MLPRegressor(hidden_layer_sizes=(100, 50, 25), activation='relu', solver='adam',
                         learning_rate_init=0.001, max_iter=1000, random_state=42)

mlp_model.fit(X_train_scaled, y_train)

# Step 4: Predictions
y_pred = mlp_model.predict(X_test_scaled)

# Step 5: Model Evaluation
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

# **Fixed RMSLE Calculation**
y_pred_safe = np.maximum(y_pred, 1e-9)  # Avoid log(0)
y_test_safe = np.maximum(y_test, 1e-9)
rmsle = np.sqrt(np.mean((np.log1p(y_pred_safe) - np.log1p(y_test_safe)) ** 2))

# **Fixed MAPE Calculation**
y_test_safe_mape = np.maximum(np.abs(y_test), 1e-2)  # Avoid division by zero
mape = np.mean(np.abs((y_test - y_pred) / y_test_safe_mape)) * 100

# Print results
print(f"SR-BPNN Regression Results:")
print(f"Mean Absolute Error (MAE): {mae:.2f}")
print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")
print(f"R-squared (R2): {r2:.2f}")
print(f"Root Mean Squared Logarithmic Error (RMSLE): {rmsle:.2f}")
print(f"Mean Absolute Percentage Error (MAPE): {mape:.2f}%")


SR-BPNN Regression Results:
Mean Absolute Error (MAE): 0.25
Mean Squared Error (MSE): 0.27
Root Mean Squared Error (RMSE): 0.52
R-squared (R2): 0.71
Root Mean Squared Logarithmic Error (RMSLE): 0.20
Mean Absolute Percentage Error (MAPE): 178.69%


In [10]:
import pandas as pd

# Assuming your file is in Excel format, use read_excel
data = pd.read_csv('/content/drive/MyDrive/lithium_pro/lith_profit6.csv')
data.head()

,Year,Virgin material value ($),Cost to recycle ($),Profit ($),Type_Direct,Type_Hydro,Type_Pyro,Cathode Scenario_LFP,Cathode Scenario_NCX,Location scenario_China-truck & tanker,Location scenario_US-train,Location scenario_US-truck,location of recycling _China,location of recycling _U.S.
0,2020,4.98,5.18,-0.20,0,0,1,0,1,0,0,1,0,1
1,2020,5.52,5.18,0.34,0,1,0,0,1,0,0,1,0,1
2,2020,4.98,4.89,0.09,0,0,1,0,1,0,1,0,0,1
3,2020,5.52,4.89,0.63,0,1,0,0,1,0,1,0,0,1
4,2020,4.98,4.22,0.76,0,0,1,0,1,1,0,0,1,0


In [11]:
X = data.drop('Profit ($)', axis=1)
y = data['Profit ($)']


print(X,y)


      Year  Virgin material value ($)  Cost to recycle ($)  Type_Direct  \
0     2020                       4.98                 5.18            0   
1     2020                       5.52                 5.18            0   
2     2020                       4.98                 4.89            0   
3     2020                       5.52                 4.89            0   
4     2020                       4.98                 4.22            0   
...    ...                        ...                  ...          ...   
6217  2050                       4.84                 5.16            0   
6218  2050                       4.84                 5.16            0   
6219  2050                       4.02                 5.16            0   
6220  2050                       4.41                 5.16            0   
6221  2050                       5.27                 5.16            0   

      Type_Hydro  Type_Pyro  Cathode Scenario_LFP  Cathode Scenario_NCX  \
0              0        

In [12]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Split the data into train, test, and validation sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)


In [13]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LassoCV
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

# Ensure X_train is a DataFrame
if isinstance(X_train, np.ndarray):
    X_train = pd.DataFrame(X_train)
    X_test = pd.DataFrame(X_test)

# Step 1: Feature Selection using Stepwise Regression (LassoCV)
lasso = LassoCV(cv=5, max_iter=10000, random_state=42)  # Increased iterations
lasso.fit(X_train, y_train)

# Select features where coefficients are non-zero
selected_indices = np.where(lasso.coef_ != 0)[0]  # Get indices of non-zero coefficients
X_train_selected = X_train.iloc[:, selected_indices]
X_test_selected = X_test.iloc[:, selected_indices]

# Step 2: Scale features for Neural Network
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_selected)
X_test_scaled = scaler.transform(X_test_selected)

# Step 3: Build & Train Backpropagation Neural Network (MLP)
mlp_model = MLPRegressor(hidden_layer_sizes=(100, 50, 25), activation='relu', solver='adam',
                         learning_rate_init=0.001, max_iter=1000, random_state=42)

mlp_model.fit(X_train_scaled, y_train)

# Step 4: Predictions
y_pred = mlp_model.predict(X_test_scaled)

# Step 5: Model Evaluation
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

# **Fixed RMSLE Calculation**
y_pred_safe = np.maximum(y_pred, 1e-9)  # Avoid log(0)
y_test_safe = np.maximum(y_test, 1e-9)
rmsle = np.sqrt(np.mean((np.log1p(y_pred_safe) - np.log1p(y_test_safe)) ** 2))

# **Fixed MAPE Calculation**
y_test_safe_mape = np.maximum(np.abs(y_test), 1e-2)  # Avoid division by zero
mape = np.mean(np.abs((y_test - y_pred) / y_test_safe_mape)) * 100

# Print results
print(f"SR-BPNN Regression Results:")
print(f"Mean Absolute Error (MAE): {mae:.2f}")
print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")
print(f"R-squared (R2): {r2:.2f}")
print(f"Root Mean Squared Logarithmic Error (RMSLE): {rmsle:.2f}")
print(f"Mean Absolute Percentage Error (MAPE): {mape:.2f}%")


SR-BPNN Regression Results:
Mean Absolute Error (MAE): 0.01
Mean Squared Error (MSE): 0.00
Root Mean Squared Error (RMSE): 0.01
R-squared (R2): 1.00
Root Mean Squared Logarithmic Error (RMSLE): 0.01
Mean Absolute Percentage Error (MAPE): 3.27%
